In [26]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, ConcatDataset
import numpy as np
from os import path
import os

In [3]:
%pwd

'/Users/sepehr/Desktop/UNI/Courses/AdvancedML/AML_HW1/Practical/Q4/src/reports'

In [4]:
%cd ../data/datasets/omniglot

/Users/sepehr/Desktop/UNI/Courses/AdvancedML/AML_HW1/Practical/Q4/src/data/datasets/omniglot


In [5]:
%cd ./Omniglot/

/Users/sepehr/Desktop/UNI/Courses/AdvancedML/AML_HW1/Practical/Q4/src/data/datasets/omniglot/Omniglot


In [29]:
from typing import Any, Callable, cast, Dict, List, Optional, Tuple
import torch

def default_loader(path: str) -> Any:
    from torchvision import get_image_backend

    if get_image_backend() == "accimage":
        return accimage_loader(path)
    else:
        return pil_loader(path)

class MultiHeadDataLloader(ImageFolder):
    def __init__(
        self,
        root: str,
        train: bool = True,
        num_tasks: int = 1,
        task_idx: int = 1,
    ):
        super().__init__(
            root,
        )
        if train:
            filtered_samples = list(filter(lambda sample: int(sample[0][-6:-4]) < 15, self.samples))
        else:
            filtered_samples = list(filter(lambda sample: int(sample[0][-6:-4]) >= 15, self.samples))
        self.samples = filtered_samples
        self.num_tasks = num_tasks
        self.task_idx = task_idx
        
        
    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        """
        Args:
            index (int): Index
        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        """
        path, target = self.samples[index]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)
            
        if self.num_tasks > 1:
            multi_target = np.zeros(self.num_tasks)-1
            multi_target[self.task_idx] = target
            target = multi_target

        return np.array(sample), target #array to hazf kon baadan

    def __len__(self) -> int:
        return len(self.samples)
    
    
class DataBatchCollator(object):
    """
    Custom collator
    """

    def __init__(self, is_training, do_augmix):
        self.is_training = is_training
        self.do_augmix = do_augmix

    def __call__(self, batch: List) -> (torch.Tensor, torch.Tensor):
        """
        :param batch:
        :return:
        """
        labels = [x[1] for x in batch]
        labels = torch.tensor(labels)

        if self.do_augmix and self.is_training:
            inputs = batch[0]
            inputs = np.array([x[0] for x in inputs])
            inputs_aug1 = np.array([x[1] for x in inputs])
            inputs_aug2 = np.array([x[2] for x in inputs])
            inputs = np.vstack([inputs, inputs_aug1, inputs_aug2])
        else:
            inputs = np.array([x[0] for x in batch])

        inputs = torch.tensor(inputs)
        inputs = inputs.permute([0, 3, 1, 2])
        return inputs, labels

In [30]:
tasks = sorted(list(filter(os.path.isdir, os.listdir(os.getcwd()))))
print( len(tasks) )
for task_idx, subdir in enumerate(tasks):
    sub_data = MultiHeadDataLloader(root = path.abspath('./'+subdir), train=True, num_tasks=len(tasks), task_idx=task_idx)
    dl = DataLoader(sub_data, batch_size=64, collate_fn=DataBatchCollator(False, False))
    for idx, (inputs, labels) in enumerate(iter(dl)):
        print(labels[:])
        break
    break
    

30


In [33]:
tasks = sorted(list(filter(os.path.isdir, os.listdir(os.getcwd()))))
print( len(tasks) )
datasets = []
for task_idx, subdir in enumerate(tasks):
    datasets.append( MultiHeadDataLloader(root = path.abspath('./'+subdir), train=True, num_tasks=len(tasks), task_idx=task_idx) )
dataset = ConcatDataset(datasets)
print(dataset)
dl = DataLoader(dataset, batch_size=1, collate_fn=DataBatchCollator(False, False), shuffle=True)
for idx, (inputs, labels) in enumerate(iter(dl)):
    print(labels[0])

30
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., 42., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1.,  5., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., 27., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., 13., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., 30., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1.,

tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  4.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., 42., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1.,  5., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         3., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., 24., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1.,  5., -1

tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         7., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1.,  2.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., 13., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., 18.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., 39.], dtype=torch.float64)
tensor([-1., -1., -1

tensor([-1., -1., -1., 19., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., 22., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         3., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1.,  6., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1.,  7., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1

tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., 17., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., 35., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., 29., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., 10.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., 14., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1

...

tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1.,  7., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1.,  0., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., 15., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., 22., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., 10., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1

tensor([-1., -1., -1., -1., 12., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., 17., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., 21., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., 13., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., 35., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1

tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., 18., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., 38.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., 53.], dtype=torch.float64)
tensor([-1., -1., 19., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1.,  9., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1

tensor([-1.,  5., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., 11., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., 22., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  5., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  7., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1

tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  9., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., 43., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., 29., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., 42., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1.], dtype=torch.float64)
tensor([-1., -1., -1

In [40]:
%cd ..

/Users/sepehr/Desktop/UNI/Courses/AdvancedML/AML_HW1/Practical/Q4/src/data/datasets/omniglot


In [42]:
os.listdir()

['Gujarati',
 'Korean',
 'Arcadian',
 'Malay_(Jawi_-_Arabic)',
 'Grantha',
 'Blackfoot_(Canadian_Aboriginal_Syllabics)',
 'Balinese',
 '.DS_Store',
 'Futurama',
 'N_Ko',
 'Burmese_(Myanmar)',
 'Anglo-Saxon_Futhorc',
 'Mkhedruli_(Georgian)',
 'Latin',
 'Braille',
 'Sanskrit',
 'Japanese_(hiragana)',
 'Tagalog',
 'Greek',
 'Ojibwe_(Canadian_Aboriginal_Syllabics)',
 'Japanese_(katakana)',
 'Early_Aramaic',
 'Hebrew',
 'Tifinagh',
 'Asomtavruli_(Georgian)',
 'Armenian',
 'Syriac_(Estrangelo)',
 'Alphabet_of_the_Magi',
 'Cyrillic',
 'Bengali',
 'Inuktitut_(Canadian_Aboriginal_Syllabics)']

In [45]:
%ls

finetune_all.yaml   from_scratch.yaml   pretrain.yaml
finetune_body.yaml  multitask1.yaml


In [51]:
import yaml

new_dict = {
    'NAMES': ['simple_head' for _ in range(30)]
}

with open('multitask1.yaml','r') as yamlfile:
    cur_yaml = yaml.safe_load(yamlfile) # Note the safe_load
    print(cur_yaml)
    cur_yaml['MODEL']['HEADS'].update(new_dict)

if cur_yaml:
    with open('multitask1.yaml','w') as yamlfile:
        yaml.safe_dump(cur_yaml, yamlfile) # Also note the safe_dump

{'EXPERIMENT_NAME': 'multitask1', 'DEBUG': True, 'DATASET': {'NAME': 'omniglot', 'DEFAULT_SIZE': '(32, 32)', 'RESIZE_SHAPE': '(32, 32)', 'INPUT_CHANNELS': 3, 'BATCH_SIZE': 32, 'PREPROCESS': False}, 'MODEL': {'PRETRAINED_PATH': '', 'BACKBONE': {'NAME': 'protonet', 'PRETRAINED_PATH': '', 'FREEZE': False}, 'NECK': {'HIDDEN_DIMS': [512, 256], 'HIDDEN_DIMS_FREEZE': [False, False]}, 'HEADS': {'NAMES': [], 'INPUT_DIMS': []}, 'SOLVER': {'TOTAL_EPOCHS': 10, 'SCHEDULER': {'NAME': 'multi_steps'}}}}


In [54]:
%ls

config.yaml       result_epoch_1.p  result_epoch_3.p  result_epoch_5.p
model.pt          result_epoch_2.p  result_epoch_4.p


In [56]:
state_dict = torch.load('./model.pt', map_location='cpu')
model.load_state_dict(state_dict['model_state'], strict=False)

NameError: name 'model' is not defined